# Sales Pipeline Summary & Risk Flagging

**User Story**: US2 — Pipeline Summary & Risk Flagging
**Persona**: Sales Manager

This notebook demonstrates how to use the Salesforce AI Assistant to generate
a team pipeline summary with at-risk deals flagged. The assistant analyzes
pipeline data grouped by AE, identifies deals with risk indicators (stalled stage,
overdue close date, no recent activity), and provides actionable recommendations.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)
- Risk thresholds configured in `config/risk_thresholds.yaml`

In [1]:
# Cell 2: Environment Check
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

✅ Environment configured
   Project: https://newfoundrydemoproject.services.ai.azure.co...
   SF Instance: https://orgfarm-181aeeadf6-dev-ed.develop.my.salesforce.com


In [ ]:
# Cell 3: Start MCP Server + Connect with MCP Client
import asyncio
import subprocess
import time

import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()

MCP_PORT = 8101
mcp_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(MCP_PORT)},
    cwd=os.path.abspath(".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

if mcp_process.poll() is not None:
    stderr = mcp_process.stderr.read().decode() if mcp_process.stderr else ""
    raise RuntimeError(f"MCP server failed to start: {stderr}")

MCP_URL = f"http://127.0.0.1:{MCP_PORT}/sse"
print(f"✅ MCP server started (PID: {mcp_process.pid}) at {MCP_URL}")


async def _connect():
    _sse_cm = sse_client(url=MCP_URL)
    read_stream, write_stream = await _sse_cm.__aenter__()
    _sess_cm = ClientSession(read_stream, write_stream)
    session = await _sess_cm.__aenter__()
    await session.initialize()
    tools_result = await session.list_tools()
    return _sse_cm, _sess_cm, session, tools_result.tools

_sse_cm, _sess_cm, mcp_session, mcp_tools = asyncio.get_event_loop().run_until_complete(_connect())

print(f"✅ MCP session connected — {len(mcp_tools)} tools available:")
for t in mcp_tools:
    print(f"   • {t.name}: {t.description[:80] if t.description else ''}")

✅ Authenticated with Azure AI Foundry


In [ ]:
# Cell 4: Create OpenAI Client + System Prompt + Chat Helper
import asyncio
import json
from pathlib import Path as _Path

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
client = project.get_openai_client()

system_prompt = _Path("../agents/sales/system_prompt.md").read_text(encoding="utf-8")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

# Convert MCP tools → OpenAI function-calling format
openai_tools = [
    {
        "type": "function",
        "function": {
            "name": t.name,
            "description": t.description or "",
            "parameters": t.inputSchema or {"type": "object", "properties": {}},
        },
    }
    for t in mcp_tools
]

async def _call_tool(name: str, arguments: dict) -> str:
    result = await mcp_session.call_tool(name, arguments)
    return "\n".join(b.text for b in result.content if hasattr(b, "text")) or ""

def chat(user_message: str, *, history: list | None = None, max_rounds: int = 10) -> list:
    """Send a message, auto-execute MCP tool calls, return updated history."""
    if history is None:
        history = [{"role": "system", "content": system_prompt}]
    history.append({"role": "user", "content": user_message})
    loop = asyncio.get_event_loop()
    for _ in range(max_rounds):
        resp = client.chat.completions.create(model=MODEL, messages=history, tools=openai_tools, tool_choice="auto")
        msg = resp.choices[0].message
        history.append(msg)
        if not msg.tool_calls:
            break
        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments) if tc.function.arguments else {}
            tool_result = loop.run_until_complete(_call_tool(tc.function.name, args))
            history.append({"role": "tool", "tool_call_id": tc.id, "content": tool_result})
    return history

def show(history: list):
    """Display the last assistant message from history."""
    for m in reversed(history):
        content = m.content if isinstance(m, dict) else m.content
        role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
        if role == "assistant" and content:
            from IPython.display import Markdown, display
            display(Markdown(content))
            return

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ {len(openai_tools)} MCP tools mapped to OpenAI functions")

In [ ]:
# Cell 5: Pipeline Query with Risk Flags

history = chat(
    "Show me my team's pipeline and flag any at-risk deals. "
    "Group the summary by AE and highlight deals that are overdue, "
    "stalled, or have no recent activity."
)
show(history)

In [ ]:
# Cell 6: Risk-Flagged Deals Deep Dive

history = chat(
    "For the at-risk deals, what specific actions should each AE take this week?",
    history=history,
)
show(history)

In [ ]:
# Cell 7: AE-Specific Follow-up
AE_NAME = "Sarah"  # Change to a specific AE name

history = chat(f"What should {AE_NAME} focus on this week?", history=history)
show(history)

In [ ]:
# Cell 8: Cleanup — close MCP session + stop server
import asyncio


async def _cleanup():
    await _sess_cm.__aexit__(None, None, None)
    await _sse_cm.__aexit__(None, None, None)

asyncio.get_event_loop().run_until_complete(_cleanup())

mcp_process.terminate()
mcp_process.wait(timeout=5)
print(f"✅ MCP session closed & server stopped (PID: {mcp_process.pid})")
print("\n--- Session Complete ---")